# 🎯 MAP Competition: DeBERTa-v3-xsmall for 6-Class Text Classification

## Overview
このノートブックは、数学的誤解（Math Misconception）分類コンペティション用のDeBERTa-v3-xsmallモデルを実装しています。

### Target Classes (6分類)
- **True_Correct**: 正解で正しい説明
- **True_Neither**: 正解だが曖昧な説明
- **True_Misconception**: 正解だが誤った概念の説明
- **False_Correct**: 不正解だが正しい概念の説明
- **False_Neither**: 不正解で曖昧な説明
- **False_Misconception**: 不正解で誤った概念の説明

### Model Architecture
- **Base Model**: microsoft/deberta-v3-xsmall
- **Parameters**: ~70M
- **Max Length**: 512 tokens
- **Evaluation Metric**: MAP@3

### Competition Format
- Input: QuestionText + MC_Answer + StudentExplanation
- Output: Top 3 predicted categories (space-separated)

## 📦 Install Required Libraries

In [ ]:
# Kaggle環境で必要なライブラリをインストール
# Kaggleでは通常、transformersとaccelerateが必要

import subprocess
import sys

def install_package(package):
    """パッケージをインストールする関数"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 必要なライブラリのインストール
try:
    import transformers
    print(f"✅ transformers already installed: {transformers.__version__}")
except ImportError:
    print("📦 Installing transformers...")
    install_package("transformers")

try:
    import accelerate
    print(f"✅ accelerate already installed: {accelerate.__version__}")
except ImportError:
    print("📦 Installing accelerate...")
    install_package("accelerate>=0.26.0")

try:
    import sentencepiece
    print(f"✅ sentencepiece already installed: {sentencepiece.__version__}")
except ImportError:
    print("📦 Installing sentencepiece...")
    install_package("sentencepiece")

print("🎉 All required libraries are ready!")

## 📚 Import Dependencies

In [ ]:
# 基本ライブラリ
import pandas as pd
import numpy as np
import os
import warnings
from pathlib import Path

# 機械学習ライブラリ
import torch
from torch.utils.data import Dataset, DataLoader

# Transformersライブラリ
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)

# Scikit-learnライブラリ
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# 警告を非表示
warnings.filterwarnings("ignore")

# デバッグ情報表示
print("🔧 Environment Information:")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Running on CPU")

# Kaggleのデータパス設定
KAGGLE_INPUT_PATH = "/kaggle/input"
if os.path.exists(KAGGLE_INPUT_PATH):
    print(f"📁 Kaggle environment detected: {KAGGLE_INPUT_PATH}")
    # MAP - Charting Student Math Misunderstandings コンペティション
    DATA_PATH = "/kaggle/input/map-charting-student-math-misunderstandings"
    print(f"🎯 Competition data path: {DATA_PATH}")
else:
    print("📁 Local environment detected")
    DATA_PATH = "."

print("✅ All libraries imported successfully!")

## 🔧 Define Dataset Class

In [ ]:
class MathMisconceptionDataset(Dataset):
    """
    Math Misconception Dataset for PyTorch
    
    6分類タスク用のカスタムデータセット:
    - True_Correct, True_Neither, True_Misconception
    - False_Correct, False_Neither, False_Misconception
    """
    
    def __init__(self, texts, labels, tokenizer, max_length=512):
        """
        Args:
            texts (list): テキストデータのリスト
            labels (list): ラベルデータのリスト
            tokenizer: DeBERTaトークナイザー
            max_length (int): 最大トークン長
        """
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # DeBERTaトークナイザーでテキストをエンコード
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

print("✅ MathMisconceptionDataset class defined successfully!")

## 📊 Load and Prepare Competition Data

In [ ]:
def load_and_prepare_data():
    """コンペデータの読み込みと前処理（6分類形式）"""
    print("=" * 60)
    print("📊 データ読み込みと前処理（6分類形式）")
    print("=" * 60)
    
    # Kaggleコンペティション: MAP - Charting Student Math Misunderstandings
    # 正確なデータパス: /kaggle/input/map-charting-student-math-misunderstandings/
    try:
        # Kaggle環境でのパス
        if os.path.exists("/kaggle/input"):
            # Kaggle環境
            kaggle_data_path = "/kaggle/input/map-charting-student-math-misunderstandings"
            train_path = f"{kaggle_data_path}/train.csv"
            test_path = f"{kaggle_data_path}/test.csv"
        else:
            # ローカル環境 - map_dataフォルダを使用
            train_path = f"{DATA_PATH}/map_data/train.csv"
            test_path = f"{DATA_PATH}/map_data/test.csv"
            
        print(f"📁 使用するパス:")
        print(f"   Train: {train_path}")
        print(f"   Test: {test_path}")
            
        train_df = pd.read_csv(train_path)
        test_df = pd.read_csv(test_path)
        
        print(f"✅ データ読み込み成功!")
        print(f"📁 Train path: {train_path}")
        print(f"📁 Test path: {test_path}")
        
    except Exception as e:
        print(f"❌ データ読み込みエラー: {e}")
        return None, None

    print(f"📈 訓練データ形状: {train_df.shape}")
    print(f"📈 テストデータ形状: {test_df.shape}")
    
    # データ確認
    print("\n📋 訓練データの列:")
    print(train_df.columns.tolist())
    
    print("\n📋 Category分布:")
    if 'Category' in train_df.columns:
        print(train_df["Category"].value_counts())
    else:
        print("❌ 'Category'列が見つかりません")
        return None, None

    # NaN値の確認と除去
    print(f"\n🧹 データクリーニング:")
    before_len = len(train_df)
    
    # 必要な列の存在確認
    required_cols = ["Category", "StudentExplanation"]
    missing_cols = [col for col in required_cols if col not in train_df.columns]
    if missing_cols:
        print(f"❌ 必要な列が不足: {missing_cols}")
        return None, None
    
    train_df = train_df.dropna(subset=required_cols)
    after_len = len(train_df)
    print(f"NaN除去: {before_len} -> {after_len} ({before_len - after_len}行削除)")

    # 強化されたテキスト特徴量の作成
    def create_enhanced_text(row):
        """Question + MC_Answer + Explanation を結合した強化テキスト"""
        question = str(row.get("QuestionText", "")) if pd.notna(row.get("QuestionText")) else ""
        mc_answer = str(row.get("MC_Answer", "")) if pd.notna(row.get("MC_Answer")) else ""
        explanation = str(row.get("StudentExplanation", "")) if pd.notna(row.get("StudentExplanation")) else ""
        
        # DeBERTa用の構造化テキスト
        enhanced_text = f"Question: {question} Selected Answer: {mc_answer} Explanation: {explanation}"
        return enhanced_text

    print("\n🔧 強化テキスト特徴量作成中...")
    train_df["enhanced_text"] = train_df.apply(create_enhanced_text, axis=1)
    test_df["enhanced_text"] = test_df.apply(create_enhanced_text, axis=1)
    
    # 6つのカテゴリの確認
    unique_categories = sorted(train_df["Category"].unique())
    print(f"\n📊 カテゴリ情報:")
    print(f"ユニークなカテゴリ数: {len(unique_categories)}")
    print("カテゴリ詳細:")
    for i, cat in enumerate(unique_categories):
        count = (train_df["Category"] == cat).sum()
        percentage = count / len(train_df) * 100
        print(f"  {i}: {cat} ({count:,}件, {percentage:.1f}%)")
    
    # サンプルテキストの表示
    print(f"\n📝 強化テキストサンプル:")
    sample_text = train_df["enhanced_text"].iloc[0]
    print(f"Length: {len(sample_text)} characters")
    print(f"Sample: {sample_text[:200]}...")
    
    return train_df, test_df

# データ読み込み実行
train_df, test_df = load_and_prepare_data()

## 🤖 Initialize DeBERTa Model and Tokenizer

In [ ]:
def prepare_model(num_labels=6, model_name="microsoft/deberta-v3-xsmall"):
    """DeBERTa-v3-xsmallモデルとトークナイザーの準備"""
    print("=" * 60)
    print(f"🤖 DeBERTaモデル準備: {model_name}")
    print("=" * 60)

    # デバイス設定（KaggleのGPU環境に最適化）
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🖥️  使用デバイス: {device}")
    
    if torch.cuda.is_available():
        print(f"🚀 GPU情報: {torch.cuda.get_device_name(0)}")
        print(f"💾 GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

    try:
        # トークナイザーの読み込み
        print("📝 DeBERTaトークナイザー読み込み中...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        # パディングトークンの確認
        print(f"🔖 パディングトークン: {tokenizer.pad_token}")
        print(f"📏 最大トークン長: {tokenizer.model_max_length}")

        # 6分類用モデルの読み込み
        print("🧠 DeBERTaモデル読み込み中...")
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels,
            torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
        )

        # デバイスに移動
        model = model.to(device)
        
        # モデル情報表示
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        
        print(f"✅ DeBERTaモデル読み込み完了!")
        print(f"📊 分類クラス数: {num_labels}")
        print(f"📈 総パラメータ数: {total_params:,}")
        print(f"🎯 訓練可能パラメータ数: {trainable_params:,}")
        print(f"💡 モデルサイズ: ~{total_params / 1e6:.1f}M parameters")

        return model, tokenizer, device

    except Exception as e:
        print(f"❌ DeBERTaモデルの読み込みに失敗: {e}")
        print("\n🔧 トラブルシューティング:")
        print("1. インターネット接続確認")
        print("2. Kaggle環境でのモデルダウンロード制限確認")
        print("3. メモリ不足の可能性確認")
        raise e

# モデル初期化の実行
if train_df is not None:
    model, tokenizer, device = prepare_model(num_labels=6)
else:
    print("❌ データが読み込まれていないため、モデル初期化をスキップします")

## ⚙️ Create Training Arguments and Configuration

In [ ]:
# Kaggle環境に最適化されたトレーニング設定
def create_training_config(device, output_dir="./deberta_model"):
    """Kaggle環境用のトレーニング設定"""
    
    # GPU利用可能性に基づく設定調整
    if torch.cuda.is_available():
        # GPU環境: より大きなバッチサイズとfp16使用
        batch_size = 8  # Kaggle GPU制限に合わせて調整
        gradient_accumulation = 4  # 実効バッチサイズ = 32
        use_fp16 = True
        epochs = 3  # Kaggle時間制限に合わせて調整
    else:
        # CPU環境: 小さなバッチサイズ
        batch_size = 2
        gradient_accumulation = 8
        use_fp16 = False
        epochs = 2
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation,
        
        # 学習率とオプティマイザー設定
        learning_rate=3e-5,  # DeBERTa推奨値
        weight_decay=0.01,
        warmup_steps=200,
        
        # 評価とロギング
        eval_strategy="steps",
        eval_steps=200,
        logging_steps=50,
        
        # モデル保存
        save_strategy="steps",
        save_steps=400,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="map3",
        greater_is_better=True,
        
        # 最適化設定
        fp16=use_fp16,
        dataloader_pin_memory=False,
        remove_unused_columns=False,
        report_to=None,  # wandbなどのロギングを無効
        
        # その他
        seed=42,
        data_seed=42,
    )
    
    print("⚙️ トレーニング設定:")
    print(f"  📊 エポック数: {epochs}")
    print(f"  📦 バッチサイズ: {batch_size}")
    print(f"  🔄 勾配蓄積: {gradient_accumulation} (実効バッチ: {batch_size * gradient_accumulation})")
    print(f"  📈 学習率: {training_args.learning_rate}")
    print(f"  🎯 FP16使用: {use_fp16}")
    print(f"  💾 出力ディレクトリ: {output_dir}")
    
    return training_args

# MAP@3メトリクス計算関数
def compute_map3_metrics(eval_pred):
    """
    MAP@3 (Mean Average Precision at 3) メトリクスの計算
    コンペティションの評価指標
    """
    predictions, labels = eval_pred
    
    # ソフトマックスで確率に変換
    predictions = torch.softmax(torch.tensor(predictions), dim=-1).numpy()
    
    map_scores = []
    for i, true_label in enumerate(labels):
        # 各サンプルの上位3つの予測を取得
        top3_indices = np.argsort(predictions[i])[::-1][:3]
        
        # MAP計算: 正解が上位3つに含まれるかチェック
        score = 0.0
        for j, pred_idx in enumerate(top3_indices):
            if pred_idx == true_label:
                score = 1.0 / (j + 1)  # 1位=1.0, 2位=0.5, 3位=0.33
                break
        map_scores.append(score)
    
    # 平均計算
    map3_score = np.mean(map_scores)
    accuracy = accuracy_score(labels, np.argmax(predictions, axis=1))
    
    return {
        "map3": map3_score,
        "accuracy": accuracy,
        "eval_loss": float(np.mean([1 - score for score in map_scores]))  # 簡易損失
    }

print("✅ トレーニング設定とメトリクス関数が定義されました！")

## 🚀 Train the Model

In [ ]:
def train_model(train_df, model, tokenizer, device):
    """DeBERTaモデルのファインチューニング実行"""
    print("=" * 60)
    print("🚀 6分類モデル ファインチューニング開始")
    print("=" * 60)
    
    # ラベルエンコーディング
    label_encoder = LabelEncoder()
    train_df["encoded_labels"] = label_encoder.fit_transform(train_df["Category"])
    
    print("🏷️ エンコードされたラベル:")
    for i, label in enumerate(label_encoder.classes_):
        count = (train_df["encoded_labels"] == i).sum()
        print(f"  {i}: {label} ({count:,}件)")
    
    # データ分割（クラス不均衡に対応）
    train_texts = train_df["enhanced_text"].tolist()
    train_labels = train_df["encoded_labels"].tolist()
    
    print(f"\n📊 データ分割実行...")
    try:
        # Stratified split（クラス比率を保持）
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts,
            train_labels,
            test_size=0.2,
            random_state=42,
            stratify=train_labels,
        )
        print("✅ Stratified split適用")
    except ValueError as e:
        print(f"⚠️ Stratified splitに失敗、通常のsplitを使用: {e}")
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts, train_labels, test_size=0.2, random_state=42
        )
    
    print(f"📈 訓練データ: {len(X_train):,}件")
    print(f"📈 検証データ: {len(X_val):,}件")
    
    # データセット作成
    print(f"\n🔧 PyTorchデータセット作成中...")
    train_dataset = MathMisconceptionDataset(
        X_train, y_train, tokenizer, max_length=512
    )
    val_dataset = MathMisconceptionDataset(
        X_val, y_val, tokenizer, max_length=512
    )
    
    print(f"✅ 訓練データセット: {len(train_dataset)}サンプル")
    print(f"✅ 検証データセット: {len(val_dataset)}サンプル")
    
    # データコレーター（動的パディング）
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    # トレーニング設定
    training_args = create_training_config(device)
    
    # Trainerの初期化
    print(f"\n🎯 Trainer初期化中...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_map3_metrics,
    )
    
    # GPU使用時のメモリ使用量表示
    if torch.cuda.is_available():
        print(f"🖥️ GPU初期メモリ使用量: {torch.cuda.memory_allocated() / 1e6:.1f} MB")
    
    # ファインチューニング実行
    print(f"\n🚀 ファインチューニング開始!")
    print(f"⏱️ 推定時間: {training_args.num_train_epochs * len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) // 60 + 1} 分")
    
    try:
        # 訓練実行
        trainer.train()
        
        # GPU使用時の最終メモリ使用量
        if torch.cuda.is_available():
            print(f"🖥️ GPU最終メモリ使用量: {torch.cuda.memory_allocated() / 1e6:.1f} MB")
        
        print(f"✅ ファインチューニング完了!")
        
        # 最終評価
        print(f"\n📊 最終モデル評価:")
        eval_results = trainer.evaluate()
        for key, value in eval_results.items():
            print(f"  📈 {key}: {value:.4f}")
        
        return trainer, label_encoder
    
    except Exception as e:
        print(f"❌ トレーニング中にエラーが発生: {e}")
        print(f"💡 メモリ不足の可能性があります。バッチサイズを下げてください。")
        raise e

# モデル訓練の実行
if 'model' in locals() and 'tokenizer' in locals() and train_df is not None:
    print("🎯 モデル訓練を開始します...")
    trainer, label_encoder = train_model(train_df, model, tokenizer, device)
    print("🎉 モデル訓練完了!")
else:
    print("❌ 必要な変数が準備されていません。前のセルを実行してください。")

## 🔮 Generate Predictions for Test Set

In [ ]:
def generate_test_predictions(trainer, tokenizer, test_df, label_encoder):
    """テストセットに対する予測生成（MAP@3形式）"""
    print("=" * 60)
    print("🔮 テストセット予測生成")
    print("=" * 60)
    
    if test_df is None or len(test_df) == 0:
        print("❌ テストデータが利用できません")
        return None
    
    print(f"📊 テストデータ: {len(test_df):,}件")
    
    # テストデータの前処理
    test_texts = test_df["enhanced_text"].tolist()
    
    # ダミーラベル（予測には使用されない）
    dummy_labels = [0] * len(test_texts)
    
    # テストデータセット作成
    print("🔧 テストデータセット作成中...")
    test_dataset = MathMisconceptionDataset(
        test_texts, dummy_labels, tokenizer, max_length=512
    )
    
    print(f"✅ テストデータセット: {len(test_dataset)}サンプル")
    
    # 予測実行
    print("🔮 予測実行中...")
    try:
        predictions = trainer.predict(test_dataset)
        print("✅ 予測完了!")
        
        # 確率に変換
        probs = torch.softmax(torch.tensor(predictions.predictions), dim=-1).numpy()
        print(f"📊 予測形状: {probs.shape}")
        
        # 各サンプルで上位3つの予測を取得
        print("🎯 TOP-3予測抽出中...")
        submission_predictions = []
        
        for i, prob in enumerate(probs):
            # 確率の高い順に上位3つのインデックスを取得
            top3_indices = np.argsort(prob)[::-1][:3]
            
            # インデックスを実際のラベル名に変換
            top3_labels = [label_encoder.classes_[idx] for idx in top3_indices]
            
            # スペース区切りで結合（コンペ要求形式）
            prediction_string = " ".join(top3_labels)
            submission_predictions.append(prediction_string)
            
            # 進捗表示（最初の5件）
            if i < 5:
                top3_probs = [prob[idx] for idx in top3_indices]
                print(f"  サンプル {i+1}: {prediction_string}")
                print(f"    確率: {[f'{p:.3f}' for p in top3_probs]}")
        
        print(f"✅ TOP-3予測抽出完了: {len(submission_predictions)}件")
        
        # 予測の統計情報
        all_predictions = " ".join(submission_predictions).split()
        from collections import Counter
        pred_counts = Counter(all_predictions)
        
        print(f"\n📈 予測統計:")
        print(f"  予測に使用されたカテゴリ数: {len(pred_counts)}")
        for category, count in pred_counts.most_common():
            percentage = count / (len(submission_predictions) * 3) * 100
            print(f"    {category}: {count}回 ({percentage:.1f}%)")
        
        return submission_predictions
    
    except Exception as e:
        print(f"❌ 予測実行中にエラー: {e}")
        if torch.cuda.is_available():
            print(f"🖥️ 現在のGPUメモリ使用量: {torch.cuda.memory_allocated() / 1e6:.1f} MB")
        raise e

# テスト予測の実行
if 'trainer' in locals() and 'label_encoder' in locals() and test_df is not None:
    print("🔮 テストセット予測を開始します...")
    test_predictions = generate_test_predictions(trainer, tokenizer, test_df, label_encoder)
    print("🎉 テスト予測完了!")
else:
    print("❌ 必要な変数が準備されていません。前のセルを実行してください。")

## 📤 Create Submission File

最終ステップとして、Kaggleコンペティションに提出するためのCSVファイルを作成します。

In [ ]:
def create_submission_file(test_df, predictions, output_path="submission.csv"):
    """提出用CSVファイルの作成"""
    print("=" * 60)
    print("📤 提出ファイル作成")
    print("=" * 60)
    
    if test_df is None or predictions is None:
        print("❌ テストデータまたは予測結果がありません")
        return None
    
    print(f"📊 提出データ: {len(predictions):,}件")
    
    # 提出用データフレーム作成
    submission_df = pd.DataFrame({
        'QuestionId_Answer': test_df.index,  # またはtest_df['QuestionId_Answer'] if available
        'Correct': predictions
    })
    
    print(f"✅ 提出データフレーム作成完了: {submission_df.shape}")
    print(f"📝 列: {list(submission_df.columns)}")
    
    # サンプル表示
    print(f"\n📋 提出データサンプル:")
    print(submission_df.head(10).to_string(index=False))
    
    # ファイル保存
    try:
        submission_df.to_csv(output_path, index=False)
        print(f"\n💾 提出ファイル保存完了: {output_path}")
        
        # ファイルサイズ確認
        import os
        file_size = os.path.getsize(output_path)
        print(f"📏 ファイルサイズ: {file_size:,} bytes ({file_size/1024:.1f} KB)")
        
        # 形式確認
        check_df = pd.read_csv(output_path)
        print(f"✅ 提出ファイル検証: {check_df.shape}")
        print(f"   必要列存在確認: {'QuestionId_Answer' in check_df.columns and 'Correct' in check_df.columns}")
        
        # 予測形式チェック
        sample_predictions = check_df['Correct'].head(5).tolist()
        print(f"🔍 予測形式サンプル:")
        for i, pred in enumerate(sample_predictions):
            pred_parts = pred.split()
            print(f"   {i+1}: {pred} (要素数: {len(pred_parts)})")
        
        return submission_df
    
    except Exception as e:
        print(f"❌ ファイル保存エラー: {e}")
        return None

# 提出ファイルの作成
if 'test_predictions' in locals() and test_df is not None:
    print("📤 提出ファイルを作成します...")
    submission_df = create_submission_file(test_df, test_predictions, "DeBERTa_v3_xsmall_submission.csv")
    
    if submission_df is not None:
        print("🎉 提出ファイル作成完了!")
        print(f"📋 最終確認:")
        print(f"   ファイル名: DeBERTa_v3_xsmall_submission.csv")
        print(f"   データ件数: {len(submission_df):,}")
        print(f"   予測形式: MAP@3 (各行に3つの予測をスペース区切り)")
        print("\n🏆 Kaggleコンペティションに提出準備完了!")
    else:
        print("❌ 提出ファイル作成に失敗しました")
else:
    print("❌ 必要なデータが準備されていません。前のセルを実行してください。")

## 🎯 Summary & Next Steps

### 📊 実行結果サマリー
このノートブックでは以下を実装しました：

1. **📚 データセット準備**: 数学の誤解分類データの前処理
2. **🤖 モデル設定**: Microsoft DeBERTa-v3-xsmall (70M parameters)
3. **⚡ 効率的な学習**: Hugging Face Transformersによる最適化
4. **📈 評価指標**: MAP@3による性能測定
5. **🔮 テスト予測**: TOP-3予測の生成
6. **📤 提出準備**: Kaggle形式のCSVファイル作成

### 🔧 使用技術
- **モデル**: `microsoft/deberta-v3-xsmall`
- **フレームワーク**: PyTorch + Transformers
- **評価**: MAP@3 (Mean Average Precision at 3)
- **最適化**: AdamW optimizer + Linear schedule

### 🚀 改善のアイデア
- データ拡張（同義語置換、パラフレーズ）
- より大きなモデル（DeBERTa-large, RoBERTa-large）
- アンサンブル手法（複数モデルの組み合わせ）
- ハイパーパラメータチューニング
- 交差検証による安定性向上

### 📝 使用方法
1. 全セルを順番に実行
2. `DeBERTa_v3_xsmall_submission.csv`が生成される
3. KaggleコンペティションにCSVファイルを提出

**🏆 Good luck with your competition!**